# Unbiased Teacher for custom dataset


This notebook is intened as an example for custom datesets with no annotations for the unlabelled images.


**Instructions**

1.   Register dataset.

To do so, add the following code to train_net.py:

```
from detectron2.data.datasets import register_coco_instances

register_coco_instances("***YOR DATASET NAME***", {}, "***PATH TO ANNOTATION JSON FILE***", "***PATH TO IMG FOLDER***")
```

Do so for both training and validation datasets. Training dataset should include both labeled and unlabeled images.

Replace this code in `unbiased-teacher/ubteacher/data/build.py`:
```
# read from pre-generated data seed
    with open(random_data_seed_path) as COCO_sup_file:
        coco_random_idx = json.load(COCO_sup_file)

labeled_idx = np.array(coco_random_idx[str(SupPercent)][str(random_data_seed)])
```
With:
`labeled_idx = np.array([i for i in range(1, num_label+1)])`

Please note that in order for this code to work, labelled images must come first in your custom dataset. If you do not know how to do that:

Load both labelled and unlabelled images in an annotation software, that can export COCO format annotations (e.g CVAT). Make sure that labelled images are loaded first. Export annotations in COCO format.

2.   Create a config file with your dataset, number of classes, annotation percent:

```
DATASETS:
  CROSS_DATASET: False
  TRAIN: ("***YOR TRAIN DATASET NAME***",)
  TEST: ("***YOR TEST DATASET NAME***",)
```
```
 ROI_HEADS:
    NAME: "StandardROIHeadsPseudoLab"
    LOSS: "FocalLoss"
    NUM_CLASSES: 1
```
```
DATALOADER:
  SUP_PERCENT: 10.0
```

A good idea would be to make a copy of `unbiased-teacher/configs/coco_supervision/faster_rcnn_R_50_FPN_sup1_run1.yaml` and replace it with your required parameters. The ones mentioned above are the ones that are required, feel free to change others aswell. SUP_PERCENT must be calculated yourself. With an incorrect SUP_PERCENT incorrect amount of images will be loaded as labeled and unlabeled.



## 0. Setup

In [ ]:
import os
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
!git clone https://github.com/gustavsma/unbiased-teacher.git


In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

## 1. Training

**BASE:**

In [ ]:
%%bash
cd /content/unbiased-teacher
python train_net.py \
      --num-gpus 1 \
      --config configs/coco_supervision/custom_base.yaml

**SSL:**

In [ ]:
%%bash
cd /content/unbiased-teacher
python train_net.py \
      --num-gpus 1 \
      --config configs/coco_supervision/custom_ssl.yaml

**RESUME:**

In [ ]:
%%bash
cd /content/unbiased-teacher
python train_net.py \
      --resume \
      --num-gpus 1 \
      --config configs/coco_supervision/custom_ssl.yaml \
      MODEL.WEIGHTS output/model_final.pth

## 2. Eval

In [ ]:
%%bash
cd /content/unbiased-teacher
python train_net.py \
      --eval-only \
      --num-gpus 1 \
      --config configs/coco_supervision/weeds_ssl.yaml \
      MODEL.WEIGHTS output/model_best.pth

## 3. Inference

In [ ]:
from detectron2.data.datasets import register_coco_instances

register_coco_instances("YOUR_TRAIN_DATASET", {}, "***PATH TO ANNOTATION JSON FILE***", "***PATH TO IMG FOLDER***")
register_coco_instances("YOUR_TEST_DATASET", {}, "***PATH TO ANNOTATION JSON FILE***", "***PATH TO IMG FOLDER***")

In [ ]:
from detectron2.config import get_cfg

cfg = get_cfg()

cfg.merge_from_file("/content/unbiased-teacher/configs/Base-RCNN-FPN.yaml")
cfg.MODEL.WEIGHTS = os.path.join("/content/unbiased-teacher/output/model_best.pth")
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.50   # Testing threshold
cfg.DATASETS.TRAIN = ("YOUR_TRAIN_DATASET", )
cfg.DATASETS.TEST = ("YOUR_TEST_DATASET", )

In [ ]:
%cd /content/gdrive/MyDrive/unbiased-teacher

from detectron2.data import DatasetCatalog
import cv2, random
from google.colab.patches import cv2_imshow
from detectron2.utils.visualizer import Visualizer
from ubteacher.engine.trainer import CustomPredictor

dataset_dicts = DatasetCatalog.get("YOUR_TEST_DATASET")
predictor = CustomPredictor(cfg)

for d in random.sample(dataset_dicts, 2):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im) 
    v = Visualizer(im[:, :, ::-1],
                   scale=1
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])